In [220]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_messages([('system','you are a helpfull ai assistant'),'{messages}'])

In [221]:
from langchain_community.llms import Ollama
model=Ollama(model='qwen3:4b')

In [232]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
store={}
def get_history(session_id):
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

In [ ]:
from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnablePassthrough,RunnableParallel
from operator import itemgetter
trimmer=trim_messages(max_tokens=500,strategy='last',allow_partial=False,include_system=True,token_counter=model)
# def chains(inp):
#     output=trimmer.invoke(inp['messages'])
#     output=prompt.invoke(output)
#     output=model.invoke(output)
#     return output
# from langchain_core.runnables import RunnableLambda
# chain = RunnableLambda(chains)
chain=RunnablePassthrough().assign(messages=itemgetter('messages')|trimmer)|prompt|RunnableParallel() |model

In [234]:
from langchain_core.runnables import RunnableWithMessageHistory,RunnablePassthrough
from langchain_core.messages import HumanMessage
app=RunnableWithMessageHistory(chain,get_history,input_messages_key='messages')
configu={'configurable':{'session_id':'chat1'}}
app.invoke({'messages':HumanMessage(content='hi my name is arun')} , config=configu)

'<think>\nOkay, the user sent a message saying "hi my name is arun". Let me think about how to respond. Since the system says I should be helpful, I need to greet them back and maybe ask how I can assist. I should keep it friendly and open-ended. Maybe say "Hello Arun! How can I assist you today?" That sounds good. Let me make sure there\'s no need for any special formatting here. The user didn\'t ask a question, so just a polite response. Alright, that\'s it.\n</think>\n\nHello Arun! How can I assist you today?'

In [ ]:
app.invoke({'messages':HumanMessage(content='what is 2+2')},config=configu)

messages=[SystemMessage(content='you are a helpfull ai assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='[HumanMessage(content=\'what is 2+2\', additional_kwargs={}, response_metadata={}), AIMessage(content=\'<think>\\nOkay, the user asked, "what is 2+2". Let me think. Well, 2 plus 2 is a basic arithmetic problem. The answer should be 4. But wait, maybe they want more details? Like, maybe they\\\'re a kid learning addition and need an explanation. Or perhaps they\\\'re testing if I know the basic math. I should confirm the answer and maybe add a simple explanation. Let me make sure there\\\'s no trick here. The question seems straightforward. So the response is 4. I\\\'ll just state that clearly.\\n</think>\\n\\nThe result of 2 + 2 is 4.\', additional_kwargs={}, response_metadata={}), HumanMessage(content=\'hi my name is arun\', additional_kwargs={}, response_metadata={}), AIMessage(content=\'<think>\\nOkay, the user sent another "hi my name is arun" messag

'<think>\nOkay, the user is asking "what is 2+2" for the third time. Let me check the conversation history. The first time, I answered 4. Then they introduced themselves as Arun, and I greeted them. The second time they asked 2+2 again, I gave the same answer. Now they\'re asking once more. It\'s possible they\'re just testing if I know the answer, or maybe they need a different explanation. But since the question is straightforward, the answer is definitely 4. I should respond clearly again. Maybe they need confirmation, so I\'ll state the answer and maybe add a friendly note to see if they need anything else.\n</think>\n\nThe result of 2 + 2 is 4. Let me know if you need help with anything else!'

In [116]:
app.invoke({'messages':'who are you'},config=configu)

Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, list[BaseMessage], or tuple[BaseMessage]. Got messages=[SystemMessage(content=\'you are a helpfull ai assistant\', additional_kwargs={}, response_metadata={}), HumanMessage(content="[HumanMessage(content=\'who are you\', additional_kwargs={}, response_metadata={})]", additional_kwargs={}, response_metadata={})].')


ChatPromptValue(messages=[SystemMessage(content='you are a helpfull ai assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content="[HumanMessage(content='who are you', additional_kwargs={}, response_metadata={})]", additional_kwargs={}, response_metadata={})])

In [235]:
app.invoke({'messages':HumanMessage(content='what is my name')},config=configu)

'<think>\nOkay, the user just asked, "what is my name". Let me see. The previous message was from the user saying "hi my name is arun", and I responded with a greeting. Now they\'re asking "what is my name". Wait, that\'s a bit confusing. If the user is asking for their own name, they might be testing if I remember it. In the history, they mentioned their name is Arun. So I should confirm that. But maybe they\'re confused or want to verify. I need to respond clearly. Let me check the system message again. The system says I should be helpful. So I should tell them their name is Arun, as they mentioned earlier. But maybe they want to make sure I remember correctly. I should state it directly. Also, maybe ask how I can help them further. Let me make sure the response is friendly and confirms their name. So the answer would be "Your name is Arun. How can I assist you today?" That seems right. No need for any other steps here. Just a straightforward reply.\n</think>\n\nYour name is Arun. Ho

In [236]:
store['chat1']

InMemoryChatMessageHistory(messages=[HumanMessage(content='hi my name is arun', additional_kwargs={}, response_metadata={}), AIMessage(content='<think>\nOkay, the user sent a message saying "hi my name is arun". Let me think about how to respond. Since the system says I should be helpful, I need to greet them back and maybe ask how I can assist. I should keep it friendly and open-ended. Maybe say "Hello Arun! How can I assist you today?" That sounds good. Let me make sure there\'s no need for any special formatting here. The user didn\'t ask a question, so just a polite response. Alright, that\'s it.\n</think>\n\nHello Arun! How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={}), AIMessage(content='<think>\nOkay, the user just asked, "what is my name". Let me see. The previous message was from the user saying "hi my name is arun", and I responded with a greeting. Now they\'re asking 

In [93]:
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.runnables import RunnablePassthrough

# Your actual chain (must expect a dict with "messages" key)
chain = RunnablePassthrough()  # example placeholder

# Your session history fetcher
def get_history(session_id: str):
    history = ChatMessageHistory()  # ✅ creates object with .messages property
    history.add_user_message("Hello")
    history.add_ai_message("Hi! How can I help you today?")
    return history

# Wrap with history
app = RunnableWithMessageHistory(chain, get_history,input_messages_key='messages')

# Call with new input and config (note the session_id)
config = {"configurable": {"session_id": "user123"}}
app.invoke({"messages": "What is the weathr?"}, config=config)


{'messages': [HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hi! How can I help you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is the weathr?', additional_kwargs={}, response_metadata={})]}